In [2]:
%run init.ipynb

matchzoo version 0.0.1
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(
    truncated_length_left=10,
    truncated_length_right=100,
    remove_stop_words=False,
    filter_low_freq=0
)

In [4]:
len(train_pack_raw)

20360

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3886.63it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:08<00:00, 2317.39it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 1445043.25it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 5009.35it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:07<00:00, 2391.53it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 346225.28it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 5318.85it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [6]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f04dad8b5f8>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f04903e5e48>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [7]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=10))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [8]:
padding_callback = mz.models.ESIM.get_default_padding_callback()

In [9]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=10
)
trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    shuffle=True,
    callback=padding_callback
)

testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    sort=False,
    shuffle=False,
    callback=padding_callback
)

In [10]:
model = mz.models.ESIM()

model.params['task'] = ranking_task
model.params['mask_value'] = 0
#model.params['embedding'] = embedding_matrix
model.params['embedding_output_dim'] = 300
model.params['embedding_input_dim'] = preprocessor.context['embedding_input_dim']

model.build()

print(model, sum(p.numel() for p in model.parameters() if p.requires_grad))

ESIM(
  (embedding): Embedding(30058, 300)
  (rnn_dropout): RNNDropout(p=0.2)
  (input_encoding): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(300, 100, bidirectional=True)
    )
  )
  (attention): BidirectionalAttention()
  (projection): Sequential(
    (0): Linear(in_features=800, out_features=200, bias=True)
    (1): ReLU()
  )
  (composition): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(200, 100, bidirectional=True)
    )
  )
  (classification): Sequential(
    (0): Dropout(p=0.2)
    (1): Linear(in_features=800, out_features=200, bias=True)
    (2): Tanh()
    (3): Dropout(p=0.2)
  )
  (out): Sequential(
    (0): Linear(in_features=200, out_features=1, bias=True)
  )
) 9901201


In [11]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [12]:
trainer.run()

[Iter-255 Loss-1.427]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6423 - normalized_discounted_cumulative_gain@5(0.0): 0.6929 - mean_average_precision(0.0): 0.6501



[Iter-510 Loss-0.392]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6387 - normalized_discounted_cumulative_gain@5(0.0): 0.688 - mean_average_precision(0.0): 0.645



[Iter-765 Loss-0.174]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6271 - normalized_discounted_cumulative_gain@5(0.0): 0.682 - mean_average_precision(0.0): 0.6436



[Iter-1020 Loss-0.106]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.622 - normalized_discounted_cumulative_gain@5(0.0): 0.662 - mean_average_precision(0.0): 0.6264



[Iter-1275 Loss-0.078]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6182 - normalized_discounted_cumulative_gain@5(0.0): 0.6646 - mean_average_precision(0.0): 0.6268



[Iter-1530 Loss-0.065]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6117 - normalized_discounted_cumulative_gain@5(0.0): 0.6689 - mean_average_precision(0.0): 0.6243



[Iter-1785 Loss-0.051]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6208 - normalized_discounted_cumulative_gain@5(0.0): 0.6773 - mean_average_precision(0.0): 0.6388



[Iter-2040 Loss-0.048]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6266 - normalized_discounted_cumulative_gain@5(0.0): 0.6779 - mean_average_precision(0.0): 0.6314



[Iter-2295 Loss-0.031]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6365 - normalized_discounted_cumulative_gain@5(0.0): 0.6857 - mean_average_precision(0.0): 0.6434



[Iter-2550 Loss-0.042]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.636 - normalized_discounted_cumulative_gain@5(0.0): 0.6753 - mean_average_precision(0.0): 0.6385

Cost time: 466.17307472229004s
